In [1]:
from data_provider.data_factory import data_provider
from utils.tools import EarlyStopping, adjust_learning_rate, visual, vali, test
from torch.utils.data import Subset
from tqdm import tqdm
from models.PatchTST import PatchTST
from models.GPT4TS import GPT4TS
from models.DLinear import DLinear
from models.TEMPO import TEMPO
# from models.T5 import T54TS
from models.ETSformer import ETSformer


import numpy as np
import torch

In [2]:
import torch.nn as nn
from torch import optim
from numpy.random import choice

import os
import time

import warnings
import matplotlib.pyplot as plt
import numpy as np

import argparse
import random
import sys

from omegaconf import OmegaConf

import torch.nn as nn
from torch import optim
from numpy.random import choice

import os
import time

import warnings
import matplotlib.pyplot as plt
import numpy as np

import argparse
import random
import sys

from omegaconf import OmegaConf

In [3]:
def get_init_config(config_path=None):
    config = OmegaConf.load(config_path)
    return config

In [4]:
from omegaconf import OmegaConf

# Define the configuration dictionary
config = {
    "datasets": {
        "Solar": {
            "root_path": "/u/dcao1/workspace/Generative-TS-train_all/csdi_data/solar_nips/",
            "data_path": "test/test.csv",
            "data_name": "Solar",
            "data": "solar",
            "lradj": "type4",
            "features": "M",
            "target": "400001",
            "embed": "timeF",
            "freq": 0,
            "percent": 100
        }
    }
}

# Create an OmegaConf object from the dictionary
cfg = OmegaConf.create(config)


In [5]:

# Save the configuration to a YAML file
with open("./configs/custom_dataset.yml", "w") as f:
    OmegaConf.save(cfg, f)

print("Configuration saved to ./configs/custom_dataset.yml")

Configuration saved to ./configs/custom_dataset.yml


In [7]:
config = {
    "description": "TEMPO",
    "checkpoints": "checkpoints/lora_revin_5domain_checkpoints_1/",
    "task_name": "long_term_forecast",
    "prompt": 1,
    "num_nodes": 1,
    "seq_len": 168,
    "pred_len": 30, #24
    "label_len": 30,
    "decay_fac": 0.5,
    "learning_rate": 0.001,
    "batch_size": 256,
    "num_workers": 0,
    "train_epochs": 10,
    "lradj": "type3",
    "patience": 5,
    "gpt_layers": 6,
    "is_gpt": 1,
    "e_layers": 3,
    "d_model": 768,
    "n_heads": 4,
    "d_ff": 768,
    "dropout": 0.3,
    "enc_in": 7,
    "c_out": 1,
    "patch_size": 16,
    "kernel_size": 25,
    "loss_func": "mse",
    "pretrain": 1,
    "freeze": 1,
    "model": "TEMPO",
    "stride": 8,
    "max_len": -1,
    "hid_dim": 16,
    "tmax": 20,
    "itr": 3,
    "cos": 1,
    "equal": 1,
    "pool": False,
    "no_stl_loss": False,
    "stl_weight": 0.001,
    "config_path": "./configs/custom_dataset.yml",
    "datasets": "ETTm1,ETTh1,ETTm2,electricity,traffic,weather",
    "target_data": "Solar",
    "use_token": 0,
    "electri_multiplier": 1,
    "traffic_multiplier": 1,
    "embed": "timeF",
    "percent": 100,
    # "model_id": 'prob_TEMPO_6_prompt_learn_168_24_100_sl336_ll168_pl24_dm768_nh4_el3_gl6_df768_ebtimeF_itr0' #"prob_TEMPO_6_prompt_learn_168_24_100_sl336_ll30_pl30_dm768_nh4_el3_gl6_df768_ebtimeF_itr0"
    "model_id": "prob_TEMPO_6_prompt_learn_168_30_100_sl336_ll30_pl30_dm768_nh4_el3_gl6_df768_ebtimeF_itr0" #"prob_TEMPO_6_prompt_learn_168_24_100_sl336_ll30_pl30_dm768_nh4_el3_gl6_df768_ebtimeF_itr0"
}

# Create an OmegaConf object from the dictionary
cfg = OmegaConf.create(config)

# Save the configuration to a YAML file
with open("configs/etth2_config.yml", "w") as f:
    OmegaConf.save(cfg, f)

print("Configuration written to config.yml")

Configuration written to config.yml


In [8]:
config = get_init_config(cfg.config_path)

In [9]:
SEASONALITY_MAP = {
   "minutely": 1440,
   "10_minutes": 144,
   "half_hourly": 48,
   "hourly": 24,
   "daily": 7,
   "weekly": 1,
   "monthly": 12,
   "quarterly": 4,
   "yearly": 1
}

In [10]:
device = torch.device('cuda:0')

In [11]:
if cfg.model == 'PatchTST':
    model = PatchTST(cfg, device)
    model.to(device)
elif cfg.model == 'DLinear':
    model = DLinear(cfg, device)
    model.to(device)
elif cfg.model == 'TEMPO':
    model = TEMPO(cfg, device)
    model.to(device)
elif cfg.model == 'T5':
    model = T54TS(cfg, device)
    model.to(device)
elif 'ETSformer' in cfg.model:
    model = ETSformer(cfg, device)
    model.to(device)
else:
    model = GPT4TS(cfg, device)


params = model.parameters()
model_optim = torch.optim.Adam(params, lr=cfg.learning_rate)

early_stopping = EarlyStopping(patience=cfg.patience, verbose=True)
if cfg.loss_func == 'mse':
    criterion = nn.MSELoss()
elif cfg.loss_func == 'smape':
    class SMAPE(nn.Module):
        def __init__(self):
            super(SMAPE, self).__init__()
        def forward(self, pred, true):
            return torch.mean(200 * torch.abs(pred - true) / (torch.abs(pred) + torch.abs(true) + 1e-8))
    criterion = SMAPE()

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(model_optim, T_max=cfg.tmax, eta_min=1e-8)

trainable params: 308736 || all params: 82207488 || trainable%: 0.38


In [12]:
model_path = os.path.join(cfg.checkpoints, cfg.model_id)
best_model_path = model_path + '/checkpoint.pth'
print(best_model_path)
model.load_state_dict(torch.load(best_model_path,  map_location=torch.device(device)), strict=False)
print("------------------------------------")

checkpoints/lora_revin_5domain_checkpoints_1/prob_TEMPO_6_prompt_learn_168_30_100_sl336_ll30_pl30_dm768_nh4_el3_gl6_df768_ebtimeF_itr0/checkpoint.pth
------------------------------------


In [13]:
cfg.data = config['datasets'][cfg.target_data].data
cfg.root_path = config['datasets'][cfg.target_data].root_path
cfg.data_path = config['datasets'][cfg.target_data].data_path
cfg.data_name = config['datasets'][cfg.target_data].data_name
cfg.features = config['datasets'][cfg.target_data].features
cfg.freq = config['datasets'][cfg.target_data].freq
cfg.target = config['datasets'][cfg.target_data].target
cfg.embed = config['datasets'][cfg.target_data].embed
cfg.percent = config['datasets'][cfg.target_data].percent
cfg.lradj = config['datasets'][cfg.target_data].lradj
if cfg.freq == 0:
    cfg.freq = 'h'
cfg.seq_len = 168 + 24
cfg.pred_len = 24 
test_data, test_loader = data_provider(cfg, 'test')

self.enc_in = 137
self.data_x = (7, 137)
test 7


In [14]:
test_data.mean_data

array([ 87.58765 ,  38.1089  ,  36.878876,  38.680717,  74.740814,
       100.72187 ,  34.929775,  34.72166 ,  38.54339 ,  36.66024 ,
        36.24879 ,  20.498371,  38.83451 ,  36.9731  ,  39.425587,
        24.803717,  38.698612,  38.37145 ,  36.375454,  36.688168,
        38.70079 ,  38.435272,  48.711697,  81.96147 ,  38.615086,
        32.350006,  36.39808 ,  38.585712,  81.54733 ,  36.545185,
        36.812992,  71.320625,  38.59133 ,  34.578987,  34.22856 ,
        30.669676,  38.383503,  27.520584,  38.10523 ,  36.47746 ,
        90.69661 ,  37.280773,  39.652153,  36.49171 ,  62.690407,
        37.48001 ,  27.493147,  34.83882 ,  38.150475,  37.91924 ,
        38.649918,  36.806263,  38.122124,  38.63643 ,  34.987503,
        38.54304 ,  38.58559 ,  34.352345,  34.68059 ,  38.409283,
        27.211773,  39.226173,  36.36521 ,  34.60476 ,  36.09768 ,
        37.850525,  39.01769 ,  38.383133,  81.14679 ,  71.17894 ,
        51.486794,  26.875301,  36.03495 , 124.58435 ,  38.810

In [15]:
# %%
import torch.distributions as dist
preds = []
trues = []
# mases = []

# Initialize accumulators for errors
total_mae = 0
total_mse = 0
n_samples = 0
itr = 0

cfg.seq_len = 168
cfg.pred_len = 24
model.eval()
# import pdb; pdb.set_trace()
with torch.no_grad():
    for i in range(len(test_data)):
        #enumerate(test_loader): #tqdm(enumerate(test_loader), total=len(test_loader)):
        data = test_data[i]
        seq_x, text_embedding, observed_mask = data[0].to(device), data[1].to(device), data[2].to(device)
        seq_x = seq_x.float().to(device)
        batch_x = seq_x[:, :cfg.seq_len].unsqueeze(-1) # 137, 168, 1
        batch_y = seq_x[:,cfg.seq_len:cfg.seq_len+cfg.pred_len]
        trues.append(batch_y.cpu().numpy())

                        
        outputs, _ = model(batch_x, 0, None, None, None) #+ model(seq_seasonal, ii) + model(seq_resid, ii)

        mu, sigma, nu = outputs[0], outputs[1], outputs[2]
        # Create the Student's t-distribution with the predicted parameters
        student_t = dist.StudentT(df=nu, loc=mu, scale=sigma)

        # Generate 30 samples for each prediction
        num_samples = 35
        probabilistic_forecasts = student_t.rsample((num_samples,))

        # The shape of probabilistic_forecasts will be (num_samples, batch_size, pred_length)
        print(probabilistic_forecasts.shape)
        preds.append(probabilistic_forecasts.cpu().numpy())


torch.Size([35, 137, 30])
torch.Size([35, 137, 30])
torch.Size([35, 137, 30])
torch.Size([35, 137, 30])
torch.Size([35, 137, 30])
torch.Size([35, 137, 30])
torch.Size([35, 137, 30])


In [16]:
trues[0]

array([[-0.81706506, -0.81706506, -0.81706506, ..., -0.81706506,
        -0.81706506, -0.81706506],
       [-0.72572196, -0.72572196, -0.72572196, ..., -0.72572196,
        -0.72572196, -0.72572196],
       [-0.71024656, -0.71024656, -0.71024656, ..., -0.71024656,
        -0.71024656, -0.71024656],
       ...,
       [-0.7138923 , -0.7138923 , -0.7138923 , ..., -0.7138923 ,
        -0.7138923 , -0.7138923 ],
       [-0.8102401 , -0.8102401 , -0.8102401 , ..., -0.8102401 ,
        -0.8102401 , -0.8102401 ],
       [-0.71037203, -0.71037203, -0.71037203, ..., -0.71037203,
        -0.71037203, -0.71037203]], dtype=float32)

In [17]:
preds[0]

array([[[-0.7650128 , -0.47545376, -1.760647  , ..., -1.2306131 ,
         -0.09866357, -1.2889092 ],
        [-0.76415837, -0.8843752 , -0.9411267 , ...,  0.17075074,
         -0.90446854, -0.82463694],
        [-0.51783204, -1.3422279 , -0.7180988 , ..., -1.7648932 ,
         -0.5568201 , -0.3918625 ],
        ...,
        [-0.641276  , -1.2405138 , -0.5358103 , ...,  0.33910656,
         -0.5380812 , -0.20042056],
        [-0.8433978 , -0.01614326, -0.6952297 , ..., -0.8741203 ,
         -0.41309783, -3.4668992 ],
        [-0.586349  , -0.6225178 , -0.95038027, ..., -0.1887055 ,
         -0.23850313, -0.47714406]],

       [[-0.8346154 , -1.3662869 , -0.9986919 , ..., -1.3542612 ,
          0.25708032, -0.393062  ],
        [-0.81588143, -1.001123  , -0.68865395, ..., -1.0375907 ,
         -0.8422663 , -0.78519213],
        [-0.38844794, -0.06995958, -1.3511779 , ..., -1.4385002 ,
         -0.25942   ,  0.64733624],
        ...,
        [-1.0257558 , -1.2554655 , -0.710295  , ..., -

In [18]:
np.array(preds).shape, np.array(trues).shape

((7, 35, 137, 30), (7, 137, 24))

In [19]:
gt_data = np.array(trues)
synthetic_data = np.array(preds)

In [20]:
synthetic_data = synthetic_data[:,:,:, :gt_data.shape[2]]
target_mask = np.ones_like(gt_data)

In [21]:
# synthetic_data = np.swapaxes(synthetic_data, -1, -2)
# gt_data = np.swapaxes(gt_data, -1, -2)
# synthetic_data.shape

In [22]:
from metrics.imputation_metrics import mse_withmask, mae_withmask, calc_quantile_CRPS, calc_quantile_CRPS_sum

In [23]:
calc_quantile_CRPS_sum(torch.Tensor(gt_data),torch.Tensor(synthetic_data),torch.Tensor(target_mask),mean_scaler=0,scaler=1)

0.8426108109323602

In [24]:
# synthetic_data = np.array(synthetic_data).reshape(synthetic_data.shape[0],-1,synthetic_data.shape[2],synthetic_data.shape[3])
# gt_data = np.array(gt_data).reshape(gt_data.shape[0],gt_data.shape[2], -1)
# target_mask = np.ones_like(gt_data)

In [25]:
unormzalized_gt_data = []
for g in gt_data:
    unormzalized_gt_data.append(np.transpose(np.transpose(g)*test_data.std_data+test_data.mean_data))
unormzalized_gt_data = np.array(unormzalized_gt_data)
unormalized_synthetic_data = []
for i in range(len(synthetic_data)):
    for j in range(len(synthetic_data[i])):
        s = synthetic_data[i][j]
        unormalized_synthetic_data.append(np.transpose(np.transpose(s)*test_data.std_data+test_data.mean_data))
unormalized_synthetic_data = np.array(unormalized_synthetic_data).reshape(synthetic_data.shape[0],-1,synthetic_data.shape[2],synthetic_data.shape[3])
unormzalized_gt_data.shape,unormalized_synthetic_data.shape

((7, 137, 24), (7, 35, 137, 24))

In [26]:
unormzalized_gt_data.shape

(7, 137, 24)

In [27]:
unormalized_synthetic_data.shape

(7, 35, 137, 24)

In [28]:
gt_data.shape

(7, 137, 24)

In [29]:
unormalized_synthetic_data = np.swapaxes(unormalized_synthetic_data, -1, -2)
unormzalized_gt_data = np.swapaxes(unormzalized_gt_data, -1, -2)


In [30]:
target_mask = np.ones_like(unormzalized_gt_data)

In [31]:
calc_quantile_CRPS_sum(torch.Tensor(unormzalized_gt_data),torch.Tensor(unormalized_synthetic_data),torch.Tensor(target_mask),mean_scaler=0,scaler=1)


0.571761733607242

In [32]:
calc_quantile_CRPS_sum(torch.Tensor(unormzalized_gt_data),torch.Tensor(unormalized_synthetic_data),torch.Tensor(target_mask),mean_scaler=0,scaler=1)


0.571761733607242

In [33]:
print("all:   CRPS_sum:")
print(calc_quantile_CRPS_sum(torch.Tensor(unormzalized_gt_data),torch.Tensor(unormalized_synthetic_data),torch.Tensor(target_mask),mean_scaler=0,scaler=1))
print("all: CRPS_sum:")
# import pdb; pdb.set_trace()
CRPS_sum_1 = calc_quantile_CRPS_sum(torch.Tensor(unormzalized_gt_data),torch.Tensor(unormalized_synthetic_data[:,:10]),torch.Tensor(target_mask),mean_scaler=0,scaler=1)
CRPS_sum_2 = calc_quantile_CRPS_sum(torch.Tensor(unormzalized_gt_data),torch.Tensor(unormalized_synthetic_data[:,10:20]),torch.Tensor(target_mask),mean_scaler=0,scaler=1)
CRPS_sum_3 = calc_quantile_CRPS_sum(torch.Tensor(unormzalized_gt_data),torch.Tensor(unormalized_synthetic_data[:,20:30]),torch.Tensor(target_mask),mean_scaler=0,scaler=1)

CRPS_sum_mean = np.mean([CRPS_sum_1, CRPS_sum_2, CRPS_sum_3])
CRPS_sum_std = np.std([CRPS_sum_1, CRPS_sum_2, CRPS_sum_3])
print("Mean CRPS_sum:", CRPS_sum_mean)
print("Standard Deviation CRPS_sum:", CRPS_sum_std)

all:   CRPS_sum:
0.571761733607242
all: CRPS_sum:
Mean CRPS_sum: 0.5814411431028131
Standard Deviation CRPS_sum: 0.00242828814261758


In [34]:
from metrics.WIS import score

In [35]:
synthetic_data = np.swapaxes(unormalized_synthetic_data, -1, -2)

labels = np.swapaxes(unormzalized_gt_data, -1, -2)

In [36]:
low_q = np.quantile(synthetic_data,0.05,axis=1)
high_q = np.quantile(synthetic_data,0.95,axis=1)
mid_q = np.quantile(synthetic_data,0.5,axis=1)

In [37]:
high_q_reshape = np.reshape(high_q,newshape=(-1,high_q.shape[-1]))
low_q_reshape = np.reshape(low_q,newshape=(-1,low_q.shape[-1]))
gt_data_reshape = np.reshape(labels,newshape=(-1,labels.shape[-1]))
print(high_q_reshape.shape,low_q_reshape.shape,gt_data_reshape.shape)

(959, 24) (959, 24) (959, 24)


In [38]:
alpha = 0.05 # to be set by competition hosts
if not(alpha > 0 and alpha <= 1):
    raise ParticipantVisibleError(f'alpha should be (0,1]. Found {alpha}')

MWIS,coverage = score(gt_data_reshape,high_q_reshape,low_q_reshape,alpha)

if not(coverage >= (1-alpha)):
    raise ParticipantVisibleError(f'Coverage was below {(1-alpha)*100}%')

In [39]:
MWIS,coverage

(228.92459874821702, 20.62148070907195)